In [ ]:
import numpy as np
import pandas as pd
from skimage.draw import ellipse_perimeter, circle_perimeter

def extract_circular_points(regions):
    circular_points = []

    for region in regions:
        # Check circularity of the region
        circularity = region.perimeter ** 2 / (4 * np.pi * region.area)

        # Define a circularity threshold (adjust as needed)
        circularity_threshold = 0.8

        if circularity > circularity_threshold:
            # Get the centroid coordinates
            centroid_row, centroid_col = region.centroid

            # Generate more results for the circular point
            additional_results = generate_additional_results(centroid_row, centroid_col)

            # Append the circular point and additional results to the list
            circular_points.append({
                'centroid_row': centroid_row,
                'centroid_col': centroid_col,
                'additional_results': additional_results
            })

    return circular_points

def generate_additional_results(row, col):
    # Generate additional results for the circular point, e.g., extract pixel values or statistics
    additional_results = {}

    # Example: Get the pixel values along the circular perimeter
    min_radius = 1
    max_radius = 15

    # Generate circular and elliptical perimeters
    perimeters = []
    for radius in range(min_radius, max_radius+1):
        circle_perimeter_row, circle_perimeter_col = circle_perimeter(int(row), int(col), radius=radius)
        ellipse_perimeter_row, ellipse_perimeter_col = ellipse_perimeter(int(row), int(col), radius, int(radius*1.5))
        perimeters.append((circle_perimeter_row, circle_perimeter_col))
        perimeters.append((ellipse_perimeter_row, ellipse_perimeter_col))

    # Extract pixel values for each perimeter
    pixel_values = []
    for perimeter_row, perimeter_col in perimeters:
        pixel_values.append((perimeter_row, perimeter_col))

    additional_results['perimeter_pixel_values'] = pixel_values

    return additional_results

In [ ]:
import numpy as np
from skimage import io, color, filters, measure
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
# from extract_circular_points import extract_circular_points

# Function to perform image segmentation
def perform_segmentation(image_path):
    # Load the image and convert it to grayscale
    img = io.imread(image_path)
    img_gray = color.rgb2gray(img)

    # Compute the distance transform
    distance = ndi.distance_transform_edt(img_gray)

    # Calculate the threshold using Otsu's method
    threshold = filters.threshold_otsu(img_gray)

    # Find peaks in the distance map
    markers = peak_local_max(distance, min_distance=10, labels=img_gray > threshold)

    # Resize markers to the same shape as the mask
    markers = np.resize(markers, img_gray.shape)

    # Perform watershed segmentation
    labeled_image = watershed(-distance, markers, mask=img_gray)

    return labeled_image

# Function to extract features from segmented regions
def extract_features_from_segmentation(labeled_image, img_gray, label, filepath):
    # Calculate the features from each segmented region
    regions = measure.regionprops(labeled_image, intensity_image=img_gray)

    # circular_points = extract_circular_points(regions)

    # # Create subplots for displaying images side by side
    # fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # # Display the grayscale image without the red color
    # axs[0].imshow(img_gray, cmap='gray')
    # axs[0].set_title('Grayscale Image')

    # # Display the original image with circular points
    # axs[1].imshow(img_gray)
    # axs[1].set_title('Original Image')

    # # Plot the circular points and perimeters
    # for circular_point in circular_points:
    #     centroid_row = circular_point['centroid_row']
    #     centroid_col = circular_point['centroid_col']
    #     additional_results = circular_point['additional_results']

    #     # Plot the centroid point as a dot
    #     axs[1].plot(centroid_col, centroid_row, 'ro', markersize=5)

    #     # Plot the circular and elliptical perimeters
    #     for perimeter_row, perimeter_col in additional_results['perimeter_pixel_values']:
    #         axs[1].plot(perimeter_col, perimeter_row, 'r-', linewidth=1)

    # # Adjust spacing between subplots
    # plt.tight_layout()

    # # Show the plot
    # plt.show()

    # # Access the circular points and additional results
    # for circular_point in circular_points:
    #     centroid_row = circular_point['centroid_row']
    #     centroid_col = circular_point['centroid_col']
    #     # additional_results = circular_point['additional_results']
    #     # print(f"Circular Point: Row={centroid_row}, Col={centroid_col}")

    features = []
    concave_points_values = []
    symmetry_values = []
    fractal_dimension_values = []

    count = 0
    for i, region in enumerate(regions):
        # print("region", region, region.perimeter, region.area, region.convex_area, region.convex_image)
        if region.area == 0 or region.perimeter == 0:
            continue
        radius = region.perimeter / (2.0 * np.pi)
        texture = np.std(img_gray[region.slice])
        perimeter = region.perimeter
        area = region.area
        smoothness = (perimeter ** 2) / area
        compactness = (perimeter ** 2) / area
        concavity = region.convex_area - area
        concave_points = len(region.convex_image) - np.sum(region.convex_image)
        IMG_SIZE = max(img_gray.shape)
        symmetry = np.sum(np.abs(img_gray[region.slice] - np.flipud(img_gray[region.slice]))) / (IMG_SIZE ** 2)
        fractal_dimension = np.log(4 * np.pi * area) / (2 * np.log(perimeter))

        if concave_points != 0:
            concave_points_values.append(concave_points)
        if concave_points == 0:
            concave_points = np.mean(concave_points_values)

        if symmetry != 0:
            symmetry_values.append(symmetry)
        if symmetry == 0:
            symmetry = np.mean(symmetry_values)

        if fractal_dimension != 0:
            fractal_dimension_values.append(fractal_dimension)
        if fractal_dimension == 0:
            fractal_dimension = np.mean(fractal_dimension_values)

        count = count+1
        feature = {
            'radius': radius,
            'texture': texture,
            'perimeter': perimeter,
            'area': area,
            'smoothness': smoothness,
            'compactness': compactness,
            'concavity': concavity,
            'concave_points': concave_points,
            'symmetry': symmetry,
            'fractal_dimension': fractal_dimension,
            'label': label,
            'path': filepath,
            'path_id': count
        }
        print("feature", feature)
        # if(i==0):
        #     print("labeled_image", labeled_image, label, feature)
        #     plt.imshow(labeled_image)
        #     plt.axis('off')  # Turn off the axis labels
        #     plt.show()
        features.append(feature)
    return features



In [ ]:
import numpy as np
from skimage import io, color, filters, measure
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
import cv2
# from extract_circular_points import extract_circular_points
# from segmentation import perform_segmentation, extract_features_from_segmentation

IMG_SIZE = 50
from google.colab import drive
drive.mount('/content/drive')

IMG_SIZE = 50

# Function to read and preprocess images
def read_and_preprocess_image(filepath):
  if filepath is not None:
    # Read the image file
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    # Resize the image to the desired size
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    # Flatten the image to a 1D array
    img = img.flatten()
    return img

def segmentation_with_image_show(labeled_image, img_gray):
    # Calculate the features from each segmented region
    regions = measure.regionprops(labeled_image, intensity_image=img_gray)

    circular_points = extract_circular_points(regions)

    # Create subplots for displaying images side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Display the grayscale image without the red color
    axs[0].imshow(img_gray, cmap='gray')
    axs[0].set_title('Grayscale Image')

    # Display the original image with circular points
    axs[1].imshow(img_gray)
    axs[1].set_title('Original Image')

    # Plot the circular points and perimeters
    for circular_point in circular_points:
        centroid_row = circular_point['centroid_row']
        centroid_col = circular_point['centroid_col']
        additional_results = circular_point['additional_results']

        # Plot the centroid point as a dot
        axs[1].plot(centroid_col, centroid_row, 'ro', markersize=5)

        # Plot the circular and elliptical perimeters
        for perimeter_row, perimeter_col in additional_results['perimeter_pixel_values']:
            axs[1].plot(perimeter_col, perimeter_row, 'r-', linewidth=1)

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()

def main():
    # Load an example image (in this case, a cropped version of one of my own photos)
    list = [
        "/content/drive/MyDrive/Colab-Notebooks/Anomaly-Detection/fiveTwelve/train/train_l_mlo_true/39672040.png"
    ]

    for filepath in list:
        print("Processing file {}".format(filepath))

        img = read_and_preprocess_image(filepath)

        # Load the grayscale image
        # Append the image and label to the lists
        labeled_image = perform_segmentation(filepath)
        img_gray = color.rgb2gray(io.imread(filepath))

        feature_array = segmentation_with_image_show(labeled_image, img_gray)

        print(feature_array)

main()


Mounted at /content/drive
Processing file /content/drive/MyDrive/Colab-Notebooks/Anomaly-Detection/fiveTwelve/train/train_l_mlo_true/39672040.png


ValueError: ignored